<a href="https://colab.research.google.com/github/NITHISHM2410/Text_Preprocessing/blob/NLP/Text%20Encoding/encoding_for_bert/TextPreprocessing_For_Bert_CustomVocabulary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

Importing a pretrained BERT Model.

In [2]:
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1")

In [3]:
bert_model.trainable = True

This Module can be used for scenarios where we need to train BERT models on our custom vocabulary instead of BERT inbuilt vocabulary as in some scenarios we don't prefer to use bert vocabulary because of its large size and predicting tokens might need a 'long' softmax layer.

In [4]:
import TextBertED as txtbert

DEFINE YOUR CUSTOM VOCABULARY IN 'vocab' PARAMETER AND MAXIMUM LENGTH OF INPUT SENTENCE IN 'max_len' PARAMETER.


In [13]:
preprocess = txtbert.BertED(
    vocab_file = True,
    vocab = "/content/words.txt",
    max_len = 40,
    max_tokens = 1_000_000

)

T0 RETURNS THE VOCABULARY : preprocess.return_vocab()

In [14]:
len(preprocess.return_vocab())

1002

PASSING FEW INPUT SENTENCES TO ENCODE BASED ON OUR CUSTOM VOCAB

In [15]:
sentences = [["I am not in danger"],["I am the danger"]]
sentences = tf.convert_to_tensor(sentences)
sentences

<tf.Tensor: shape=(2, 1), dtype=string, numpy=
array([[b'I am not in danger'],
       [b'I am the danger']], dtype=object)>

In [16]:
results = preprocess(sentences)
results

{'input_word_ids': <tf.Tensor: shape=(2, 40), dtype=int32, numpy=
 array([[  1, 508,  31,   7, 795,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0],
        [  1, 508,   2, 795,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0]], dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(2, 40), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(2, 40), dtype=int32, numpy=
 array([[0, 0, 

The Above format is the standard format for any bert model. This module also inherits Keras Layer,so it can also be directly used as preprocessing layer

In [17]:
input = tf.keras.Input(shape = (1,),batch_size = 32,dtype = tf.string)
output = txtbert.BertED(vocab = '/content/words.txt',max_len = 100,vocab_file = True)(input) # Our layer
output = bert_model(output)['pooled_output']
output = tf.keras.layers.Dense(5,activation = 'softmax')(output)
model = tf.keras.Model(inputs = [input],outputs = output)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(32, 1)]                    0         []                            
                                                                                                  
 bert_ed_3 (BertED)          {'input_word_ids': (None,    0         ['input_1[0][0]']             
                             100),                                                                
                              'input_mask': (None, 100)                                           
                             , 'input_type_ids': (None,                                           
                              100)}                                                               
                                                                                              

Passing a Input to the model

In [19]:
sentences.shape

TensorShape([2, 1])

In [20]:
outputs = model(sentences)

In [21]:
outputs.shape

TensorShape([2, 5])

DECODING USING THE SAME MODULE (Integer token -> String)

In [22]:
integer_token_sentence = tf.convert_to_tensor([[100,102,104,912],[256,413,235,154]])

In [24]:
preprocess.back_to_string(integer_token_sentence)

<tf.Tensor: shape=(2, 4), dtype=string, numpy=
array([[b'now', b'any', b'work', b'thank'],
       [b'white', b'note', b'story', b'three']], dtype=object)>